In [ ]:
from google.colab import drive
drive.mount('/drive', force_remount=True)

Mounted at /drive


In [ ]:
import csv
import pandas as pd
import math
from tqdm import tqdm

In [ ]:
def resetDictionary(lineup):
    lineup.clear()
    lineup["Players"]=("OELO","DELO","POS","P1","P2","P3","P4","P5","TEAM","offpoints","defpoints","offPOS","defPOS")

In [ ]:
def format_players(team, row):
    players = list(map(lambda x: row[f"{team}_PLAYER_ID_{x+1}"], range(5)))
    for i in range(len(players)):
        if not math.isnan(players[i]):
            players[i]=int(players[i])
    players.sort()
    players = map(lambda p: str(p), players)
    return "-".join((players))

In [ ]:
names = pd.read_csv (r"/drive/MyDrive/basketballData/BasketballELO/PlayerList08.csv", low_memory=False)
names=names.set_index('PERSON_ID').to_dict()['DISPLAY_FIRST_LAST']
def playersNames(input):
  ids = list(map(int, input.split("-")))
  return str(names[ids[0]]+"-"+names[ids[1]]+"-"+names[ids[2]]+"-"+names[ids[3]]+"-"+names[ids[4]])

In [ ]:
nba18pbp = pd.read_csv (r"/drive/MyDrive/basketballData/BasketballELO/events_2017-2018_pbp.csv", low_memory=False)
nba18pbp['HOME_players'] = nba18pbp.apply(lambda r: format_players('HOME',r), axis=1)
nba18pbp['AWAY_players'] = nba18pbp.apply(lambda r: format_players('AWAY',r), axis=1)

In [ ]:
games = nba18pbp.groupby('GAME_ID')[nba18pbp.keys()].apply(lambda g: list(map(tuple, g.values.tolist()))).to_dict()
for game in games.keys():
  games[game] = pd.DataFrame(games[game], columns = nba18pbp.keys())

In [ ]:
teamID = {}
for game in tqdm(games.keys()):
  for index, row in games[game].iterrows():
    if row["PLAYER1_ID"] not in names.keys() and row["PLAYER1_ID"] not in teamID.keys():
      if not isinstance(row["VISITORDESCRIPTION"], float):
        if row["AWAY_TEAM"].lower() in row["VISITORDESCRIPTION"].lower():
          teamID[row["PLAYER1_ID"]] = row["AWAY_TEAM"]
      if not isinstance(row["HOMEDESCRIPTION"], float):
        if row["HOME_TEAM"].lower() in row["HOMEDESCRIPTION"].lower():
          teamID[row["PLAYER1_ID"]] = row["HOME_TEAM"]
  if len(teamID.keys()) == 30:
    break

  6%|▌         | 73/1230 [00:01<00:31, 36.97it/s]


In [ ]:
def check(row):
  reb = {}
  if not math.isnan(row["REBOUND_PLAYER_ID"]):
    if row["PLAYER1_ID"] not in teamID.keys():
      if row["PLAYER1_ID"] not in reb:
          reb[row["PLAYER1_ID"]] = 0
      if row["PLAYER1_TEAM_NICKNAME"] == row["HOME_TEAM"]:
        if reb[row["PLAYER1_ID"]] < row["REBOUND_DEFENSIVE_COUNT"]:
          return "HOME"
        else:
          return ""
        reb[row["PLAYER1_ID"]] = row["REBOUND_DEFENSIVE_COUNT"]
      else:
        if reb[row["PLAYER1_ID"]] < row["REBOUND_DEFENSIVE_COUNT"]:
          return "AWAY"
        else:
          return ""
        reb[row["PLAYER1_ID"]] = row["REBOUND_DEFENSIVE_COUNT"]
    else:
      if teamID[row["PLAYER1_ID"]] == row["HOME_TEAM"]:
        return "HOME"
      else:
        return "AWAY"
  elif not math.isnan(row["FREE_THROW_MADE"]) and row["FREE_THROW_MADE"] == 1:
    if row["FREE_THROW_NUM"]==row["FREE_THROW_TOTAL"]:
      if row["PLAYER1_TEAM_NICKNAME"] == row["HOME_TEAM"]:
        return "HOME"
      else:
        return "AWAY"
  elif not math.isnan(row["JUMP_BALL_RETRIEVED_PLAYER_ID"]):
    if row["PLAYER3_TEAM_NICKNAME"] == row["HOME_TEAM"]:
      return "HOME"
    else:
      return "AWAY"
  elif not math.isnan(row["SHOT_MADE"]) and int(row["SHOT_MADE"])==1:
    if row["PLAYER1_TEAM_NICKNAME"] == row["HOME_TEAM"]:
      return "AWAY"
    else:
      return "HOME"
  elif not math.isnan(row["TEAM_TURNOVER_COUNT"]):
    if row["PLAYER1_ID"] not in teamID.keys():
      if row["PLAYER1_TEAM_NICKNAME"] == row["HOME_TEAM"]:
        return "AWAY"
      else:
        return "HOME"
    else:
      if teamID[row["PLAYER1_ID"]] == row["HOME_TEAM"]:
        return "AWAY"
      else:
        return "HOME"
  return ""

In [ ]:
for game in tqdm(games.keys()):
  pos = []
  i=0
  j=0
  a=0
  for index, row in games[game].iterrows():
    pos.append(check(row))
    i+=1
  pos[0]=pos[1]
  currentPos = ""
  for i in range(len(pos)):
    if pos[i] != "":
      currentPos = pos[i]
    else:
      pos[i] = currentPos
  games[game]["CURRENT_POS"]=pos

100%|██████████| 1230/1230 [00:44<00:00, 27.58it/s]


In [ ]:
pointsPerPos=1.2465222012000134

In [ ]:
def setUpPlayers(lineUpData,row,):
  if row["AWAY_players"] not in lineUpData.keys():
    lineUpData[row["AWAY_players"]] = ["",0,0,0,0,0,1000,1000]
  if row["HOME_players"] not in lineUpData.keys():
    lineUpData[row["HOME_players"]] = ["",0,0,0,0,0,1000,1000]
  if lineUpData[row["AWAY_players"]][0]=="":
    if str(row["PLAYER1_ID"]) in row["AWAY_players"]and row["PLAYER1_TEAM_NICKNAME"]==row["AWAY_TEAM"]:
      lineUpData[row["AWAY_players"]][0]=row["AWAY_TEAM"]
    elif str(row["PLAYER2_ID"]) in row["AWAY_players"]and row["PLAYER2_TEAM_NICKNAME"]==row["AWAY_TEAM"]:
      lineUpData[row["AWAY_players"]][0]=row["AWAY_TEAM"]
    elif str(row["PLAYER3_ID"]) in row["AWAY_players"]and row["PLAYER3_TEAM_NICKNAME"]==row["AWAY_TEAM"]:
      lineUpData[row["AWAY_players"]][0]=row["AWAY_TEAM"]
  if lineUpData[row["HOME_players"]][0]=="":
    if str(row["PLAYER1_ID"]) in row["HOME_players"]and row["PLAYER1_TEAM_NICKNAME"]==row["HOME_TEAM"]:
      lineUpData[row["HOME_players"]][0]=row["HOME_TEAM"]
    elif str(row["PLAYER2_ID"]) in row["HOME_players"]and row["PLAYER2_TEAM_NICKNAME"]==row["HOME_TEAM"]:
      lineUpData[row["HOME_players"]][0]=row["HOME_TEAM"]
    elif str(row["PLAYER3_ID"]) in row["HOME_players"]and row["PLAYER3_TEAM_NICKNAME"]==row["HOME_TEAM"]:
      lineUpData[row["HOME_players"]][0]=row["HOME_TEAM"]
    return lineUpData

In [ ]:
def calc(teamO,teamD,score,lineUpData,row,pos):
  lineUpData[row[f"{teamO}_players"]][3] += score
  lineUpData[row[f"{teamD}_players"]][4] += score
  if lineUpData[row[f"{teamO}_players"]][4]>0:
    lineUpData[row[f"{teamO}_players"]][5] = lineUpData[row[f"{teamO}_players"]][3] /lineUpData[row[f"{teamO}_players"]][4]
  lineUpData[row[f"{teamO}_players"]][2] += pos
  # print("score ",teamO,": ",score," in ",pos," calced ",(score/pos/(pointsPerPos*2)))
  score/=pos

  eloDiff=lineUpData[row[f"{teamD}_players"]][7] - lineUpData[row[f"{teamO}_players"]][6]
  expectedScored = (( 1.0 / (1 + 1.0 *  math.pow(10, 1.0 * (eloDiff) / 400))))
  addD = 32*pos*(expectedScored-(score/(pointsPerPos*2)))
  addO = -addD
  # print(eloDiff," Elo diff")
  # print(expectedScored," expected")
  # print(addD ," addD")
  lineUpData[row[f"{teamO}_players"]][6] +=addO
  lineUpData[row[f"{teamD}_players"]][7] +=addD
  return lineUpData

In [ ]:
lineUpData = {}
for j in range(5):
  i=0
  numGames=0
  points=0
  for game in tqdm(games.keys()):
    currentHomeLineUp = ""
    currentAwayLineUp = ""
    currentHomeScore = 0
    currentAwayScore = 0
    lineupHomePoints = 0
    lineupAwayPoints = 0
    curPos = ""
    time = 0
    aPos = 0
    hPos = 0
    preRow = 0
    for index, row in games[game].iterrows():
      setUpPlayers(lineUpData,row,)

      if row["AWAY_SCORE"] - currentAwayScore<0 or row["AWAY_SCORE"] - currentAwayScore>5:
        currentAwayScore = row["AWAY_SCORE"]
      if row["HOME_SCORE"] - currentHomeScore<0 or row["HOME_SCORE"] - currentHomeScore>5:
        currentHomeScore = row["HOME_SCORE"]
      lineupAwayPoints +=  row["AWAY_SCORE"] - currentAwayScore
      lineupHomePoints +=  row["HOME_SCORE"] - currentHomeScore


      if row["HOME_players"] != currentHomeLineUp:
        currentHomeLineUp=row["HOME_players"]
        if aPos>0 and hPos>0:
          # print(lineupAwayPoints," points ", aPos," pos")
          # print(lineupHomePoints," points ", hPos," pos")
          calc("HOME","AWAY",lineupHomePoints,lineUpData,preRow,hPos)
          calc("AWAY","HOME",lineupAwayPoints,lineUpData,preRow,aPos)
          lineupAwayPoints=0
          lineupHomePoints=0
          aPos = 0
          hPos = 0

      if row["AWAY_players"] != currentAwayLineUp:
        currentAwayLineUp=row["AWAY_players"]
        if aPos>0 and hPos>0:
          calc("HOME","AWAY",lineupHomePoints,lineUpData,preRow,hPos)
          calc("AWAY","HOME",lineupAwayPoints,lineUpData,preRow,aPos)
          lineupAwayPoints=0
          lineupHomePoints=0
          aPos = 0
          hPos = 0



      if (row["TIME"] - time)<0:
        time = row["TIME"]

      lineUpData[row["AWAY_players"]][1] += (row["TIME"] - time)/60
      lineUpData[row["HOME_players"]][1] += (row["TIME"] - time)/60
      time = row["TIME"]

      if curPos !="HOME"and row["CURRENT_POS"] =="HOME":
        hPos+=1
        # print()
        # print(row["AWAY_SCORE"] - currentAwayScore," Away Score")
        # print(row["HOME_SCORE"] - currentHomeScore," HOME Score")
        i+=1

      if curPos !="AWAY"and row["CURRENT_POS"] =="AWAY":
        # print()
        # print(row["AWAY_SCORE"] - currentAwayScore," Away Score")
        # print(row["HOME_SCORE"] - currentHomeScore," HOME Score")
        aPos+=1
        i+=1

      preRow = row
      currentHomeScore = row["HOME_SCORE"]
      currentAwayScore = row["AWAY_SCORE"]
      # print(row["CURRENT_POS"]," Home Score ",currentHomeScore," Away Score ",currentAwayScore,)
      curPos = row["CURRENT_POS"]
    points+=int(row["AWAY_SCORE"])+int(row["HOME_SCORE"])
    numGames+=1
    # if numGames > 100:
    #   break
  posPerGame = i/(numGames*2)
  pointPerGame = points/(numGames*2)
  pointsPerPos = pointPerGame/posPerGame

print()
print(posPerGame)
print(pointPerGame)
print(pointsPerPos)


100%|██████████| 1230/1230 [01:22<00:00, 14.92it/s]


85.29715447154472
106.32479674796748
1.2465222012000134


In [ ]:
lineUpDF=[]
for key, value in lineUpData.items():
  lineUpDF.append((key, *value))
lineUpDF  = pd.DataFrame(lineUpDF)
lineUpDF.columns = ("Players","Team", "Time","Pos","Points Scored","Points Allowed","Diff","OELO","DELO")
lineUpDF['TOTAL ELO'] = lineUpDF.apply(lambda r:(r["OELO"]+r["DELO"])/2, axis=1)
lineUpDF['Players'] = lineUpDF['Players'].apply(playersNames)
lineUpDF = lineUpDF[lineUpDF["Time"]>15][lineUpDF["Pos"]>10]

<ipython-input-138-7cf23c735c09>:8: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  lineUpDF = lineUpDF[lineUpDF["Time"]>15][lineUpDF["Pos"]>10]


In [ ]:
lineUpDF.describe()
#lineUpDF.sort_values(by=['TOTAL ELO'],ascending=False).head(50)

,Time,Pos,Points Scored,Points Allowed,Diff,OELO,DELO,TOTAL ELO
count,6797.000000,6797.000000,6797.000000,6797.000000,6797.000000,6797.000000,6797.000000,6797.000000
mean,79.651096,137.968221,172.114168,169.997058,1.158941,1012.238851,999.521380,1005.880116
std,202.672199,363.928295,452.449333,429.244369,0.756314,114.825995,117.429711,78.978256
min,15.000000,15.000000,0.000000,0.000000,0.000000,538.918258,361.531985,671.612224
25%,22.250000,40.000000,45.000000,45.000000,0.785714,937.260303,931.158614,956.095885
50%,34.750000,60.000000,75.000000,75.000000,1.013699,1008.389193,1005.613876,1006.349120
75%,67.750000,115.000000,150.000000,145.000000,1.329114,1082.384883,1075.628574,1057.177125
max,5657.166667,9930.000000,12635.000000,11830.000000,14.000000,1660.590669,1424.860217,1303.807706


In [ ]:
lineUpDF.sort_values(by=['TOTAL ELO'],ascending=False).head(20)

,Players,Team,Time,Pos,Points Scored,Points Allowed,Diff,OELO,DELO,TOTAL ELO
15825,Carmelo Anthony-Raymond Felton-Corey Brewer-Ru...,Thunder,22.333333,35,80,0,0.000000,1282.670329,1324.945083,1303.807706
5685,Andre Iguodala-Nick Young-JaVale McGee-Kevon L...,Warriors,16.750000,20,85,30,3.148148,1583.691880,1022.909439,1303.300660
16462,Marvin Williams-Kemba Walker-Jeremy Lamb-Willy...,Hornets,20.250000,40,135,50,2.872340,1640.035190,956.306467,1298.170828
76,Shaun Livingston-Kevin Durant-Nick Young-Steph...,Warriors,29.916667,45,140,75,1.917808,1627.885043,951.947019,1289.916031
10113,Trevor Ariza-Gerald Green-PJ Tucker-Eric Gordo...,Rockets,31.500000,55,125,25,5.000000,1318.995850,1246.864640,1282.930245
12474,DeMarre Carroll-Allen Crabbe-Joe Harris-D'Ange...,Nets,23.166667,45,95,25,3.800000,1287.402089,1266.319168,1276.860629
4198,Mike Conley-Marc Gasol-Tyreke Evans-Ben McLemo...,Grizzlies,20.666667,30,60,10,6.000000,1193.337447,1351.683021,1272.510234
2230,Zaza Pachulia-Kevin Durant-Stephen Curry-Klay ...,Warriors,99.333333,185,335,175,1.914286,1168.739379,1345.727701,1257.233540
6998,Trevor Ariza-Chris Paul-Eric Gordon-Ryan Ander...,Rockets,30.666667,40,125,75,1.785714,1542.197519,960.952838,1251.575178
14555,Goran Dragic-James Johnson-Rodney McGruder-Jus...,,16.750000,35,70,5,14.000000,1227.213750,1271.965287,1249.589518


In [ ]:
teamsData = lineUpDF.groupby("Team")
teamDF = {}
cutOff = 15
for t in tqdm(teamID.keys()):
  team = teamID[t]
  teamData = teamsData.get_group(team)
  teamData = teamData[teamData["Time"]>cutOff]
  totalTime = teamData["Time"].sum()
  teamDF[team] = [0,0,0]
  for index, row in teamData.iterrows():
    teamDF[team][0] += row["OELO"] *(row["Time"]/totalTime)
    teamDF[team][1] += row["DELO"] *(row["Time"]/totalTime)
  teamDF[team][2] = (teamDF[team][1]+teamDF[team][0])/2
temp = []
for key, value in teamDF.items():
  temp.append((key, *value))
teamDF  = pd.DataFrame(temp)
teamDF.columns = ("Team","OELO","DELO","TOTAL ELO")

100%|██████████| 30/30 [00:00<00:00, 83.05it/s]


In [ ]:
teamsData.get_group("Rockets").sort_values(by=['Time'],ascending=False)

,Players,Team,Time,Pos,Points Scored,Points Allowed,Diff,OELO,DELO,TOTAL ELO
5437,Trevor Ariza-Chris Paul-PJ Tucker-James Harden...,Rockets,1334.666667,2290,2780,2745,1.012750,907.682609,1013.889519,960.786064
66,Trevor Ariza-Eric Gordon-Ryan Anderson-James H...,Rockets,1086.916667,1975,2690,2240,1.200893,1021.343706,985.655637,1003.499672
41,Trevor Ariza-Chris Paul-Ryan Anderson-James Ha...,Rockets,931.916667,1830,2335,2135,1.095216,937.217996,1061.500825,999.359411
511,Trevor Ariza-PJ Tucker-Eric Gordon-James Harde...,Rockets,799.416667,1365,1765,1670,1.056886,975.853966,1055.007363,1015.430665
5042,Chris Paul-PJ Tucker-Luc Mbah a Moute-James Ha...,Rockets,656.166667,1060,1470,1205,1.221945,1136.753695,1034.550540,1085.652118
...,...,...,...,...,...,...,...,...,...,...
3184,Nene-Trevor Ariza-PJ Tucker-Eric Gordon-Luc Mb...,Rockets,15.833333,30,55,25,2.200000,1004.696526,1061.208774,1032.952650
2129,PJ Tucker-Eric Gordon-Ryan Anderson-Bobby Brow...,Rockets,15.666667,25,30,45,0.666667,1045.900752,823.950501,934.925627
13621,Joe Johnson-Chris Paul-Gerald Green-PJ Tucker-...,Rockets,15.583333,20,45,0,0.000000,1191.269224,1286.296298,1238.782761
14169,Joe Johnson-Trevor Ariza-Ryan Anderson-Luc Mba...,Rockets,15.500000,25,50,15,3.333333,1143.689291,1029.977693,1086.833492


In [ ]:
teamDF.sort_values(by=['TOTAL ELO'],ascending=False)

,Team,OELO,DELO,TOTAL ELO
22,76ers,1004.801191,1055.419711,1030.110451
12,Jazz,1026.112733,1033.542701,1029.827717
11,Timberwolves,1035.811957,1022.376547,1029.094252
29,Thunder,1043.534241,1011.574037,1027.554139
7,Trail Blazers,1023.290894,1029.101117,1026.196006
18,Nuggets,1033.190212,1013.158965,1023.174588
5,Pelicans,1008.967250,1034.502294,1021.734772
1,Warriors,1032.772548,1009.776169,1021.274359
0,Rockets,1028.946816,1013.395038,1021.170927
19,Clippers,1030.640791,1006.873339,1018.757065


In [ ]:
lineUpDF.sort_values(by=['Diff'],ascending=False).head(50)

,Players,Team,Time,Pos,Points Scored,Points Allowed,Diff,OELO,DELO,TOTAL ELO
14555,Goran Dragic-James Johnson-Rodney McGruder-Jus...,,16.750000,35,70,5,14.000000,1227.213750,1271.965287,1249.589518
18799,Vince Carter-Willie Cauley-Stein-Buddy Hield-D...,,15.083333,25,45,5,11.250000,1131.381033,1177.747456,1154.564244
9385,Arron Afflalo-D.J. Augustin-Marreese Speights-...,Magic,16.250000,20,55,5,11.000000,1312.519575,1186.511730,1249.515653
6118,Arron Afflalo-D.J. Augustin-Nikola Vucevic-Eva...,Magic,23.666667,35,50,5,10.000000,1056.442734,1274.612920,1165.527827
18960,Ian Mahinmi-Jodie Meeks-Tomas Satoransky-Mike ...,Wizards,19.166667,35,45,5,9.000000,1064.296999,1248.863654,1156.580326
1331,Ersan Ilyasova-Marco Belinelli-Kent Bazemore-D...,Hawks,17.750000,40,45,5,9.000000,1020.121583,1327.204556,1173.663070
17464,Lou Williams-Austin Rivers-C.J. Williams-Montr...,,29.333333,50,90,10,9.000000,1103.305718,1269.454585,1186.380152
16108,Paul Millsap-Wilson Chandler-Will Barton-Trey ...,Nuggets,15.500000,30,45,5,9.000000,1105.804405,1238.733340,1172.268873
9887,Pau Gasol-Tony Parker-Kawhi Leonard-Davis Bert...,,15.166667,20,35,5,8.750000,1110.975698,1183.466325,1147.221012
11180,Jordan Clarkson-Julius Randle-Larry Nance Jr.-...,Lakers,27.416667,45,70,10,8.750000,1023.933257,1356.426097,1190.179677


In [ ]:
numPos = 100
per100pos = pd.DataFrame(lineUpDF[lineUpDF['Pos']>numPos]["Players"])
per100pos["Team"] = lineUpDF[lineUpDF['Pos']>numPos]["Team"]
per100pos["Points Scored"] = lineUpDF[lineUpDF['Pos']>numPos].apply(lambda r:(r["Points Scored"]/(r["Pos"]+1)*100), axis=1)
per100pos["Points Allowed"] = lineUpDF[lineUpDF['Pos']>numPos].apply(lambda r:(r["Points Allowed"]/(r["Pos"]+1)*100), axis=1)
per100pos["Margin"] = lineUpDF[lineUpDF['Pos']>numPos].apply(lambda r:(r["Diff"]/(r["Pos"]+1)*100), axis=1)

In [ ]:
per100pos.sort_values(by=['Margin'],ascending=False)#.describe()#.sort_values(by=['Margin'],ascending=False)

,Players,Team,Points Scored,Points Allowed,Margin
3348,Jameer Nelson-Dante Cunningham-DeMarcus Cousin...,Pelicans,141.346154,100.961538,1.413462
404,Paul Millsap-Will Barton-Gary Harris-Nikola Jo...,Nuggets,152.884615,114.423077,1.306706
1445,Goran Dragic-James Johnson-Wayne Ellington-Kel...,Heat,161.061947,114.159292,1.288496
8251,DeAndre Jordan-Austin Rivers-C.J. Williams-Jam...,Clippers,123.076923,100.961538,1.183432
7697,Chris Paul-Ryan Anderson-Luc Mbah a Moute-Jame...,Rockets,133.962264,111.320755,1.175108
...,...,...,...,...,...
4097,Eric Bledsoe-John Henson-Khris Middleton-Tony ...,Bucks,124.236818,119.056429,0.096532
123,Thaddeus Young-Darren Collison-Bojan Bogdanovi...,Pacers,122.760291,118.805488,0.083397
3960,Kyle Lowry-Serge Ibaka-DeMar DeRozan-Jonas Val...,Raptors,128.161312,114.969241,0.076196
410,Jeff Teague-Taj Gibson-Jimmy Butler-Andrew Wig...,Timberwolves,129.922879,119.897172,0.055833


In [ ]:
teamData=per100pos.groupby("Team").sum()
teamData.sort_values(by=['Margin'],ascending=False)

<ipython-input-44-1cd51a944886>:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  teamData=per100pos.groupby("Team").sum()


,Points Scored,Points Allowed,Margin
Team,,,
,37376.715911,37410.796283,2009.996447
Suns,16958.895161,18219.787130,702.761085
Bucks,14723.511708,12844.440484,625.397008
Grizzlies,13413.041580,13566.982290,619.860325
Pelicans,13741.991523,13446.198998,595.394556
Hornets,15162.246345,15079.732195,571.883109
Hawks,14002.023619,14396.764229,571.016728
Wizards,13925.298779,13790.751961,533.322638
Lakers,13306.007466,13388.464225,531.233314
